In [338]:
import pandas as pd
import numpy as np

In [339]:
book_path = 'movies_metadata'
fix_book_path = 'movies_metadata_fixed' 

In [341]:
with open(f'{book_path}.csv', 'r', encoding='utf-8') as f, open(f'{fix_book_path}.csv', 'w', encoding='utf-8') as g:
    content = f.read().replace('\n ','\t ')
    g.write(content)
    print("Fixed file {0}.csv".format(fix_book_path))

Fixed file movies_metadata_fixed.csv


In [342]:
book   = pd.read_csv(f'{fix_book_path}.csv', low_memory=False)

In [343]:
print(book.info(verbose=True))
book

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45463 non-null  bool   
 1   belongs_to_collection  4491 non-null   object 
 2   budget                 45463 non-null  int64  
 3   genres                 45463 non-null  object 
 4   homepage               7779 non-null   object 
 5   id                     45463 non-null  int64  
 6   imdb_id                45446 non-null  object 
 7   original_language      45452 non-null  object 
 8   original_title         45463 non-null  object 
 9   overview               44509 non-null  object 
 10  popularity             45463 non-null  float64
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45458,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1
45459,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3
45460,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6
45461,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0


In [344]:
# book['overview'] = book['overview'].str.replace('\t ','\n ')

In [345]:
book = book.drop_duplicates(subset=['id'])
book.shape

(45433, 24)

In [346]:
testing = book[['genres','id']].copy()

In [347]:
import json
testing['genres'] = testing['genres'].str.replace("'", '"').apply(json.loads)

In [348]:
s = testing.explode('genres')
df_json = s.to_json(orient='records')
finaldf = pd.json_normalize(json.loads(df_json), meta=['id'])

finaldf = finaldf.drop('genres', axis=1).dropna()
finaldf.columns = ['movie_id', 'genre_id', 'genre_name']
finaldf['genre_id'] = finaldf['genre_id'].astype(int)

In [349]:
print(finaldf.info(verbose=True))
finaldf

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91015 entries, 0 to 93454
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   movie_id    91015 non-null  int64 
 1   genre_id    91015 non-null  int32 
 2   genre_name  91015 non-null  object
dtypes: int32(1), int64(1), object(1)
memory usage: 2.4+ MB
None


,movie_id,genre_id,genre_name
0,862,16,Animation
1,862,35,Comedy
2,862,10751,Family
3,8844,12,Adventure
4,8844,14,Fantasy
...,...,...,...
93450,439050,10751,Family
93451,111109,18,Drama
93452,67758,28,Action
93453,67758,18,Drama


In [350]:
genre_table = finaldf.groupby(['genre_id','genre_name']).size().reset_index().rename(columns = {0: 'count'})
genre_table

,genre_id,genre_name,count
0,12,Adventure,3490
1,14,Fantasy,2309
2,16,Animation,1931
3,18,Drama,20244
4,27,Horror,4671
5,28,Action,6592
6,35,Comedy,13176
7,36,History,1398
8,37,Western,1042
9,53,Thriller,7619


In [351]:
genre_table = genre_table.drop('count', axis=1)

In [352]:
filepath = "./proj/Genre"
genre_table.to_csv(f'{filepath}.csv', index=False)

In [353]:
join_table = finaldf.drop('genre_name', axis=1)

In [354]:
filepath = "./proj/join_movie_genre"
join_table.to_csv(f'{filepath}.csv', index=False)

In [355]:
movie_only = book[['adult','id','title', 'overview', 'runtime','release_date',\
                   'vote_average', 'original_language', 'original_title', 'popularity']].copy()

In [356]:
movie_only.columns = ['adult','movie_id','title','description','duration','release_date','vote','o_lan', 'o_tit', 'pop']

In [357]:
movie_only['duration'] = movie_only['duration'].fillna(value=0).astype(int).astype(str) + ' minutes'

In [358]:
def adultcheck(x):
    if x:
        return 'C18'
    else:
        return 'P'

movie_only['age_restriction'] = movie_only['adult'].apply(adultcheck)

In [359]:
movie_only

,adult,movie_id,title,description,duration,release_date,vote,o_lan,o_tit,pop,age_restriction
0,False,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",81 minutes,1995-10-30,7.7,en,Toy Story,21.946943,P
1,False,8844,Jumanji,When siblings Judy and Peter discover an encha...,104 minutes,1995-12-15,6.9,en,Jumanji,17.015539,P
2,False,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,101 minutes,1995-12-22,6.5,en,Grumpier Old Men,11.712900,P
3,False,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",127 minutes,1995-12-22,6.1,en,Waiting to Exhale,3.859495,P
4,False,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,106 minutes,1995-02-10,5.7,en,Father of the Bride Part II,8.387519,P
...,...,...,...,...,...,...,...,...,...,...,...
45458,False,439050,Subdue,Rising and falling between a man and woman.,90 minutes,NaN,4.0,fa,رگ خواب,0.072051,P
45459,False,111109,Century of Birthing,An artist struggles to finish his work while a...,360 minutes,2011-11-17,9.0,tl,Siglo ng Pagluluwal,0.178241,P
45460,False,67758,Betrayal,"When one of her hits goes wrong, a professiona...",90 minutes,2003-08-01,3.8,en,Betrayal,0.903007,P
45461,False,227506,Satan Triumphant,"In a small town live two brothers, one a minis...",87 minutes,1917-10-21,0.0,en,Satana likuyushchiy,0.003503,P


In [360]:
movie_table = movie_only.drop(['adult', 'vote', 'o_lan', 'o_tit', 'pop'], axis=1)
movie_table = movie_table[['movie_id','title','description','age_restriction','duration', 'release_date']]

In [361]:
movie_table

,movie_id,title,description,age_restriction,duration,release_date
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",P,81 minutes,1995-10-30
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,P,104 minutes,1995-12-15
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,P,101 minutes,1995-12-22
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",P,127 minutes,1995-12-22
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,P,106 minutes,1995-02-10
...,...,...,...,...,...,...
45458,439050,Subdue,Rising and falling between a man and woman.,P,90 minutes,NaN
45459,111109,Century of Birthing,An artist struggles to finish his work while a...,P,360 minutes,2011-11-17
45460,67758,Betrayal,"When one of her hits goes wrong, a professiona...",P,90 minutes,2003-08-01
45461,227506,Satan Triumphant,"In a small town live two brothers, one a minis...",P,87 minutes,1917-10-21


In [362]:
filepath = "./proj/Movie"
movie_table.to_csv(f'{filepath}.csv', index=False)

In [368]:
movie_retest = movie_table

In [369]:
movie_retest['len'] = movie_retest['title'].apply(len)

In [370]:
movie_retest.sort_values(by='len')

,movie_id,title,description,age_restriction,duration,release_date,len
16032,49207,R,The R of the title stands for the young protag...,P,99 minutes,2010-04-22,1
33862,263471,I,A bodybuilder ends up falling madly in love wi...,P,170 minutes,2015-01-04,1
26014,53459,F,A group of teachers must defend themselves fro...,P,79 minutes,2010-09-07,1
16166,36373,M,"Remake of the 1931 original. In the city, some...",P,88 minutes,1951-03-01,1
22986,45514,8,8 shorts centered around 8 themes directed by ...,P,100 minutes,2008-10-23,1
...,...,...,...,...,...,...,...
30362,68816,Will Our Heroes Be Able to Find Their Friend W...,"A rich businessman is fed up with work, family...",P,130 minutes,1968-12-19,88
36823,270936,Ellen ten Damme: As I Was Wondering Where This...,Documentary about Dutch artist Ellen ten Damme...,P,66 minutes,2007-04-19,90
36854,316111,Riuscirà l'avvocato Franco Benenato a sconfigg...,NaN,P,0 minutes,1971-08-18,101
7241,10338,Those Magnificent Men in Their Flying Machines...,Star studded comedy about a early 20th century...,P,138 minutes,1965-06-03,104
